In [1]:
import math

def calculate_cast_distance(rod, reel, line, hook, bait, bobber):
    # 1. Calculate CastWeight (in kg)
    cast_weight = bobber['weight'] + bobber['sinker_weight'] + hook['weight'] + bait['weight']
    
    # 2. Calculate CastWeightFactor
    min_cast_weight = rod['min_cast_weight']
    max_cast_weight = rod['max_cast_weight']
    mid_weight = (min_cast_weight + max_cast_weight) / 2
    
    if cast_weight <= min_cast_weight:
        cast_weight_factor = 0.85
    elif cast_weight >= max_cast_weight:
        cast_weight_factor = 1.0
    else:
        if cast_weight <= mid_weight:
            lower_bound = 0.85
            upper_bound = 1.0
            weight_range = mid_weight - min_cast_weight
            cast_weight_factor = lower_bound + (upper_bound - lower_bound) * (cast_weight - min_cast_weight) / weight_range
        else:
            cast_weight_factor = 1.0

    # 3. Reel Factor
    reel_factor = reel['friction']

    # 4. Calculate CastTackleFactor (Windage factor for bobber and bait)
    cast_tackle_factor = bobber['windage'] * bait['windage']

    # 5. Calculate LineFactor
    min_thickness = 0.05
    max_thickness = 0.35
    line_thickness = line['thickness']
    
    if line_thickness <= min_thickness:
        line_factor = 1.0
    elif line_thickness >= max_thickness:
        line_factor = 0.8
    else:
        lower_bound = 1.0
        upper_bound = 0.8
        thickness_range = max_thickness - min_thickness
        line_factor = lower_bound + (upper_bound - lower_bound) * (line_thickness - min_thickness) / thickness_range

    # 6. Calculate CastDistanceMax
    cast_distance_max = rod['max_cast_distance'] * cast_weight_factor * reel_factor * cast_tackle_factor * line_factor
    
    # 7. Apply safeguard for minimum cast distance
    cast_distance_max = max(rod['length'] * 2, cast_distance_max)

    return round(cast_distance_max, 2)

def calculate_throwing_distance_indicator(rod, reel, line, bait):
    # 1. Calculate kRodLength
    k_rod_length = rod['length'] * 2.5

    # 2. Determine kRodHardness based on rod hardness type
    rod_hardness_type = rod['hardness_type']
    if rod_hardness_type == "[竿硬度]L(软)":
        k_rod_hardness = 1.1
    elif rod_hardness_type == "[竿硬度]M(中)":
        k_rod_hardness = 1.0
    else:
        k_rod_hardness = 0.9

    # 3. Calculate kReelSize
    k_reel_size = (reel['spool_size'] - 1000) / 5000 + 1

    # 4. Calculate kLineD
    k_line_d = 1.2 - (line['diameter'] * 0.004)

    # 5. Determine kLineMaterial based on line material type
    line_material = line['material']
    if line_material == "[主线]尼龙线":
        k_line_material = 0.9
    elif line_material == "[主线]编织线":
        k_line_material = 1.1
    else:
        k_line_material = 1.0

    # 6. Determine kBaitWeight based on bait weight
    bait_weight = bait['weight'] * 1000  # Convert kg to g
    if bait_weight < 7:
        k_bait_weight = 0.9
    elif bait_weight > 28:
        k_bait_weight = 0.95
    else:
        k_bait_weight = 1.1

    # 7. Calculate cast potential
    cast_potential = 2 * k_rod_length * k_rod_hardness * k_reel_size * k_line_d * k_line_material * k_bait_weight

    # # 8. Calculate throwing distance indicator
    # throwing_distance_base = 10  # Assume ThrowingDistanceBase is 10 for calculation
    # throwing_distance_indicator = math.log(cast_potential) / math.log(throwing_distance_base)

    # 9. Clamp the value between 0 and 1
    # throwing_distance_indicator = max(0, min(1, throwing_distance_indicator))

    # return round(throwing_distance_indicator, 2)
    return cast_potential

# Example input data for throwing distance indicator calculation
def main():
    rods = [
        {
            'id': 3012001,
            'max_cast_distance': 50,   # in meters
            'min_cast_weight': 0.005,  # in kg (5g)
            'max_cast_weight': 0.015,  # in kg (15g)
            'length': 2.4,             # in meters
            'hardness_type': "[竿硬度]L(软)"
        },
        {
            'id': 3012002,
            'max_cast_distance': 23,   # in meters
            'min_cast_weight': 0.005,  # in kg (5g)
            'max_cast_weight': 0.012,  # in kg (12g)
            'length': 3.4,             # in meters
            'hardness_type': "[竿硬度]M(中)"
        }
        # Add more rods as needed...
    ]

    reel = {
        'friction': 1,
        'spool_size': 3000
    }

    line = {
        'thickness': 0.25,
        'diameter': 0.25,
        'material': "[主线]编织线"
    }

    hook = {
        'weight': 0.001  # in kg (1g)
    }

    bait = {
        'weight': 0.01,   # in kg (10g)
        'windage': 0.95
    }

    bobber = {
        'weight': 0.005,       # in kg (5g)
        'sinker_weight': 0.001,  # in kg (1g)
        'windage': 0.95
    }

    # Calculate cast distance and throwing distance indicator for each rod
    for rod in rods:
        cast_distance = calculate_cast_distance(rod, reel, line, hook, bait, bobber)
        throwing_distance_indicator = calculate_throwing_distance_indicator(rod, reel, line, bait)
        print(f"Rod ID: {rod['id']}, Maximum Cast Distance: {cast_distance} meters, Throwing Distance Indicator: {throwing_distance_indicator}")

if __name__ == "__main__":
    main()


Rod ID: 3012001, Maximum Cast Distance: 39.11 meters, Throwing Distance Indicator: 10.724239680000002
Rod ID: 3012002, Maximum Cast Distance: 17.99 meters, Throwing Distance Indicator: 13.8115208
